In [92]:
import pandas as pd
import requests
from bs4 import BeautifulSoup


In [147]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.162 Safari/537.36'}



# Finding total pages in the website

listing_url = 'https://merojob.com/category/it-telecommunication/?page=1'
response = requests.get(listing_url, headers = headers)
soup = BeautifulSoup(response.text, 'lxml')

pagination = soup.find('ul', class_= 'pagination')

if pagination:
    # find the tatal page numbers except 'next' or 'previous'
    pages = pagination.find_all('li')
    page_numbers = []

    for page in pages:
        try:
            num = int(page.text.strip())   
            page_numbers.append(num)          
        except:
            pass
            
    max_page = max(page_numbers) if page_numbers else 1
else:
    max_page = 1


In [223]:

job_list = []
final = pd.DataFrame()

#including base url because href only includes page address
base_url = "https://merojob.com"
    
# we are not using range(max_page+1) because it will start from 0 by default
for i in range(1, (max_page + 1)):
    webpage = requests.get('https://merojob.com/category/it-telecommunication/?page={}'.format(i), headers = headers).text
    
    soup = BeautifulSoup(webpage, 'lxml')

    # In website from where data is scraped, jobs are under classname "job-card"
    #.text is only used on single items so using loop to seperate single item from list
    #.strip() is used to remove any trailing whitespaces and namespaces
    
    job_cards = soup.find_all('h1', class_= 'media-heading')
    specific_cards = soup.find_all('hw')

    # if not job_cards:
    #     print('No jobs found on this page, stopping.')
    #     break

    for job in job_cards:
        link_tag = job.find('a', href= True)
        if link_tag:
            job_url = base_url + link_tag['href']
            # print(f"Visiting: {job_url}")

            job_detail_resp = requests.get(job_url, headers = headers)
            job_soup = BeautifulSoup(job_detail_resp.text, 'lxml')
        
            
            if job_soup:

                job_data = {}
                
                
                head_info = job_soup.find('div', class_= 'media-body mt-3')
                
                job_title_info = head_info.find_all('h1')

                #there are more than one element in head_info so    
                for title in job_title_info:
                    job_data['Job Title'] = title.text.strip()

                company_name_info = head_info.find_all('a')
                
                for name in company_name_info:
                    job_data['Company Name'] = name.text.strip()

            

                table_info = job_soup.find_all('table', class_= 'table')

                for table in table_info:
                    for row in table.find_all('tr'):
                         cells = row.find_all(['th', 'td'])
                         if cells:
                             key = cells[0].text.strip()
                             value = cells[2].text.strip().replace('\xa0', ' ').replace('\n', ' ').strip()
                             job_data[key] = value
                
                # print(job_data)             
                job_list.append(job_data.copy())
                # print(job_list)

# print(f"Total number of jobs:{len(job_list)}")
df = pd.DataFrame(job_list)
df.to_csv('merojob_jobs.csv')


        
